In [1]:
import numpy as np 
import pandas as pd 
import re
import os
import glob
import pickle
from sklearn.model_selection import train_test_split 
import librosa
import random

In [2]:
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import concatenate
from keras.layers import Reshape
from keras import optimizers
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf
from keras.layers.recurrent import LSTM


Using TensorFlow backend.


In [7]:
INPUT_FOLDER = "../../data_kaggle/freesound/input"
# INPUT_FOLDER = "../input/"
print(os.listdir(INPUT_FOLDER))

['train_curated.csv', 'train_noisy.csv', 'sample_submission.csv', 'train_noisy', 'train_curated', 'test']


In [6]:
num_freq = 128
len_div = 256

In [7]:
n_hidden = 300
n_filter = 8

conv_list = []

inputs = Input(shape=(num_freq,len_div,1 ), name='input')

conv1 = Conv2D(n_filter, (17, 1),activation='relu',padding='same',name='conv1')(inputs)
pool1 = MaxPooling2D((17, 1),strides=(4, 1),padding='same',name='pool1')(conv1)
norm1 = BatchNormalization(axis=-1, name='norm1')(pool1)
reshape1 = Reshape([-1, len_div])(norm1)
conv_list.append(reshape1)

conv2 = Conv2D(n_filter, (11, 1),activation='relu',padding='same',name='conv2')(inputs)
pool2 = MaxPooling2D((11, 1),strides=(2, 1),padding='same',name='pool2')(conv2)
norm2 = BatchNormalization(axis=-1, name='norm2')(pool2)
reshape2 = Reshape([-1, len_div])(norm2)
conv_list.append(reshape2)

conv3 = Conv2D(n_filter, (7, 1),activation='relu',padding='same',name='conv3')(inputs)
pool3 = MaxPooling2D((7, 1),strides=(4, 1),padding='same',name='pool3')(conv3)
norm3 = BatchNormalization(axis=-1, name='norm3')(pool3)
reshape3 = Reshape([-1, len_div])(norm3)
conv_list.append(reshape3)

concat = concatenate(conv_list, name='concat', axis=1)

lstm = LSTM(n_hidden, return_sequences=False, name='LSTM')(concat)
drop = Dropout(rate=0.05)(lstm)
dense = Dense(80, name='dense1')(drop)
pred = Activation('softmax',name='pred')(dense)

adam = optimizers.Adam(lr=0.0001)

model = Model(inputs=inputs, outputs=pred)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 128, 256, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 256, 8)  144         input[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 128, 256, 8)  64          input[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 32, 256, 8)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool3 (Max

In [9]:
def tf_one_sample_positive_class_precisions(y_true, y_pred) :
    num_samples, num_classes = y_pred.shape
    
    # find true labels
    pos_class_indices = tf.where(y_true > 0) 
    
    # put rank on each element
    retrieved_classes = tf.nn.top_k(y_pred, k=num_classes).indices
    sample_range = tf.zeros(shape=tf.shape(tf.transpose(y_pred)), dtype=tf.int32)
    sample_range = tf.add(sample_range, tf.range(tf.shape(y_pred)[0], delta=1))
    sample_range = tf.transpose(sample_range)
    sample_range = tf.reshape(sample_range, (-1,num_classes*tf.shape(y_pred)[0]))
    retrieved_classes = tf.reshape(retrieved_classes, (-1,num_classes*tf.shape(y_pred)[0]))
    retrieved_class_map = tf.concat((sample_range, retrieved_classes), axis=0)
    retrieved_class_map = tf.transpose(retrieved_class_map)
    retrieved_class_map = tf.reshape(retrieved_class_map, (tf.shape(y_pred)[0], num_classes, 2))
    
    class_range = tf.zeros(shape=tf.shape(y_pred), dtype=tf.int32)
    class_range = tf.add(class_range, tf.range(num_classes, delta=1))
    
    class_rankings = tf.scatter_nd(retrieved_class_map,
                                          class_range,
                                          tf.shape(y_pred))
    
    #pick_up ranks
    num_correct_until_correct = tf.gather_nd(class_rankings, pos_class_indices)

    # add one for division for "presicion_at_hits"
    num_correct_until_correct_one = tf.add(num_correct_until_correct, 1) 
    num_correct_until_correct_one = tf.cast(num_correct_until_correct_one, tf.float32)
    
    # generate tensor [num_sample, predict_rank], 
    # top-N predicted elements have flag, N is the number of positive for each sample.
    sample_label = pos_class_indices[:, 0]   
    sample_label = tf.reshape(sample_label, (-1, 1))
    sample_label = tf.cast(sample_label, tf.int32)
    
    num_correct_until_correct = tf.reshape(num_correct_until_correct, (-1, 1))
    retrieved_class_true_position = tf.concat((sample_label, 
                                               num_correct_until_correct), axis=1)
    retrieved_pos = tf.ones(shape=tf.shape(retrieved_class_true_position)[0], dtype=tf.int32)
    retrieved_class_true = tf.scatter_nd(retrieved_class_true_position, 
                                         retrieved_pos, 
                                         tf.shape(y_pred))
    # cumulate predict_rank
    retrieved_cumulative_hits = tf.cumsum(retrieved_class_true, axis=1)

    # find positive position
    pos_ret_indices = tf.where(retrieved_class_true > 0)

    # find cumulative hits
    correct_rank = tf.gather_nd(retrieved_cumulative_hits, pos_ret_indices)  
    correct_rank = tf.cast(correct_rank, tf.float32)

    # compute presicion
    precision_at_hits = tf.truediv(correct_rank, num_correct_until_correct_one)

    return pos_class_indices, precision_at_hits

def tf_lwlrap(y_true, y_pred):
    num_samples, num_classes = y_pred.shape
    pos_class_indices, precision_at_hits = (tf_one_sample_positive_class_precisions(y_true, y_pred))
    pos_flgs = tf.cast(y_true > 0, tf.int32)
    labels_per_class = tf.reduce_sum(pos_flgs, axis=0)
    weight_per_class = tf.truediv(tf.cast(labels_per_class, tf.float32),
                                  tf.cast(tf.reduce_sum(labels_per_class), tf.float32))
    sum_precisions_by_classes = tf.zeros(shape=(num_classes), dtype=tf.float32)  
    class_label = pos_class_indices[:,1]
    sum_precisions_by_classes = tf.unsorted_segment_sum(precision_at_hits,
                                                        class_label,
                                                       num_classes)
    labels_per_class = tf.cast(labels_per_class, tf.float32)
    labels_per_class = tf.add(labels_per_class, 1e-7)
    per_class_lwlrap = tf.truediv(sum_precisions_by_classes,
                                  tf.cast(labels_per_class, tf.float32))
    out = tf.cast(tf.tensordot(per_class_lwlrap, weight_per_class, axes=1), dtype=tf.float32)
    return out

In [10]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=[tf_lwlrap])

In [13]:
datagen = ImageDataGenerator(
           rotation_range=0,
           width_shift_range=64,
           height_shift_range=0,
           shear_range=0,
           zoom_range=0,
           horizontal_flip=False,
           vertical_flip=False)

In [14]:
batch_size = 32

epochs = 20
for n in range(epochs):
    print('epoch No.{}'.format(n))
    print('learning with curated data')
    with open('preprocessed_dataset/train_arr_0.pickle', 'rb') as f:
        X_train0 = pickle.load(f)
        y_train0 = pickle.load(f)
    with open('preprocessed_dataset/train_arr_1.pickle', 'rb') as f:
        X_train1 = pickle.load(f)
        y_train1 = pickle.load(f)
    X_train = np.vstack([X_train0, X_train1])
    y_train = np.vstack([y_train0, y_train1])
    
    with open('preprocessed_dataset/val_arr_0.pickle', 'rb') as f:
        X_val0 = pickle.load(f)
        y_val0 = pickle.load(f)
    with open('preprocessed_dataset/val_arr_1.pickle', 'rb') as f:
        X_val1 = pickle.load(f)
        y_val1 = pickle.load(f)
    X_val = np.vstack([X_val0, X_val1])
    y_val = np.vstack([y_val0, y_val1])
     
    model.fit(X_train, y_train,batch_size=batch_size,epochs=2,validation_data=(X_val, y_val))    
    
    pick = random.sample(range(8),8)
    for i, m in enumerate(pick):
        print('learning with noisy data No.{}'.format(i))
        with open('preprocessed_dataset/noisy_train_arr_{}.pickle'.format(m), 'rb') as f:
            X_train = pickle.load(f)
            y_train = pickle.load(f)
        pick_val = random.sample(range(2),1)[0]
        with open('preprocessed_dataset/noisy_val_arr_{}.pickle'.format(pick_val), 'rb') as f:
            X_val = pickle.load(f)
            y_val = pickle.load(f)
        model.fit(X_train, y_train,
                  batch_size=batch_size,
                  epochs=2,
                  validation_data=(X_val, y_val))

epoch No.0
learning with curated data
Train on 7550 samples, validate on 1827 samples
Epoch 1/2
7550/7550 [==============================] - 135s 18ms/step - loss: 4.8790 - tf_lwlrap: 0.1407 - val_loss: 4.6542 - val_tf_lwlrap: 0.1525
Epoch 2/2
7550/7550 [==============================] - 131s 17ms/step - loss: 4.6630 - tf_lwlrap: 0.1859 - val_loss: 4.5414 - val_tf_lwlrap: 0.1849
learning with noisy data No.0
Train on 5878 samples, validate on 5866 samples
Epoch 1/2
5878/5878 [==============================] - 126s 21ms/step - loss: 5.3208 - tf_lwlrap: 0.1185 - val_loss: 5.0753 - val_tf_lwlrap: 0.1167
Epoch 2/2
5878/5878 [==============================] - 126s 21ms/step - loss: 5.1480 - tf_lwlrap: 0.1493 - val_loss: 4.9989 - val_tf_lwlrap: 0.1371
learning with noisy data No.1
Train on 5856 samples, validate on 5750 samples
Epoch 1/2
5856/5856 [==============================] - 124s 21ms/step - loss: 5.0274 - tf_lwlrap: 0.1633 - val_loss: 4.9876 - val_tf_lwlrap: 0.1462
Epoch 2/2
5856/585

KeyboardInterrupt: 

In [15]:
model.save('20190511-2model.h5', include_optimizer=False)